In [21]:
import os
import json
import re
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
from sklearn.metrics import confusion_matrix, roc_curve, precision_recall_curve, auc
import pandas as pd
from tqdm import tqdm  # Import tqdm

# 加载数据

In [22]:
test_data_path = "./datasets/train_test/test0819.jsonl"
model_path = r"D:\Pretrained_models\Qwen\Qwen2-1___5B-Instruct"

In [23]:
import chardet


def load_jsonl(path):
    with open(path, 'r', encoding="utf-8") as file:
        data = [json.loads(line) for line in file]
        return data


test_data = load_jsonl(test_data_path)

true_data = [d for d in test_data if d['label'] == True]
false_data = [d for d in test_data if d['label'] == False]
print(f"total_count:{len(test_data)}, true_count:{len(true_data)}, false_count:{len(false_data)}")

total_count:2349, true_count:1182, false_count:1167


# 加载模型

In [24]:
def load_model(model_path, checkpoint_path='', device='cuda'):
    # 加载tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
    # 加载模型
    model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True, torch_dtype=torch.bfloat16)
    # 加载LoRA权重
    if checkpoint_path:
        model = PeftModel.from_pretrained(model, checkpoint_path).to(device)

    return model, tokenizer


model, tokenizer = load_model(model_path, device='cuda')
model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((1536,), eps=1e-06)
    (rotary_emb): Qw

# 推理函数
定义一个推理函数 predict, 用传入的模型和一条对话文本来进行欺诈文本分类的推理
1. 既要测试原始模型，也要测试微调后的模型，并且可能会微调多个版本，所以把model和tokenizer作为参数传入
2. 由于模型预测结果的不确定性，在使用json加载解析response时可能会报一场，需要加一个safe_loads保护

In [27]:
def safe_loads(text, default_value=None, debug = False):
    print(f"safe_loads and text is {text}") if debug else None
    json_string = re.sub(r'^```json\n(.*)\n```$', r'\1', text.strip(), flags=re.DOTALL)
    try:
        return json.loads(json_string)
    except json.JSONDecodeError as e:
        print(f"invalid json: {json_string}")
        return default_value


def predict(model, tokenizer, content, device='cuda', debug=False):
    prompt = f"下面是一段对话文本，请分析对话内容是否有诈骗风险，只以json格式输出你的判断结果(is_fraud:true/false)。\n\n{content}"

    # 返回的格式是 [batch_size, sequence_length]，其中 batch_size 是输入的样本数量，sequence_length 是输入文本的 token 数量（经过 tokenizer 处理后的文本
    inputs = tokenizer.apply_chat_template(
        [{
            "role": "user",
            "content": prompt,
        }],
        add_generation_prompt=True,
        tokenize=True,
        return_tensors="pt",
        return_dict=True).to(device)

    default_response = {"is_fraud":False}
    gen_kwargs = {"max_new_tokens":2048, "do_sample":True, "top_k":1}

    # 将模型加载到显存
    model.to(device)

    with torch.no_grad():
        outputs = model.generate(**inputs, **gen_kwargs)
        outputs = outputs[:, inputs['input_ids'].shape[1]:]
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        return safe_loads(response, default_response)

In [28]:
predict(model,tokenizer, "李强: 很好。接下来请你在APP上充值50元并完成一个简单的购物任务，完成后你会看到返利进入你的账户。\n王丽: 我已经完成了充值和第一个任务，返利已经到账，谢谢。\n李强: 你做得很好！现在，我们有一个更高价值的任务，你只需充值100元并完成任务，这次返利会更高。\n王丽: 好的，我马上充值完成任务。\n李强: 再接再厉，现在我们有一个特别高回报的任务，只需你再充值10000元，完成任务后，你将得到更多返利。\n王丽: 我已经充值10000元并完成任务，但为什么这次我没有收到返利？\n李强: 系统提示你的账户存在异常，目前资金被冻结。你需要再充值5000元解冻账户，这样才能提现全部金额。",device='cuda')

{'is_fraud': True}

# 定义评测方法
评测主要是对所有数据的预测结果进行是否正确的统计与分析,最终根据自己的侧重点来计算出一两个指标用于评估性能

定义test_data来进行分类预测,并返回所有数据的真是标签和预测标签

In [ ]:
result = pd.read_json("./datasets/train_test/test0819.jsonl", lines=True)
result.head()

In [29]:
def run_test(model, tokenizer, test_data, device='cuda', debug=False):
    real_labels = []
    pred_labels = []


    for i, item in tqdm(enumerate(test_data), total=len(test_data), desc="Processing", unit="item"):
        if debug:
            print(item['input'])

        dialog_input = item['input']
        real_label = item['label']

        prediction = predict(model, tokenizer, dialog_input, device=device, debug=debug)

        if debug:
            print(f"prediction is {prediction}")

        pred_label = prediction['is_fraud']

        real_labels.append(real_label)
        pred_labels.append(pred_label)

        if debug and i % (len(test_data) // 20 + 1) == 0:
            print(f"percent: {(i * 100) / len(test_data):.2f}%")

    return real_labels, pred_labels

In [30]:
test_data[10]

{'input': '发言人4: 第二个是历来旅游交通都是景区所谓的增修也好，创新也好，一个主要的一个手段。包括我们一直在统计2销的时候，这个旅游消费，这个旅游交通的消费一直统计到2销面的。其实所以说整体来讲，这个东西不太会有大的一个变化，因为其实坦白讲不太有什么动力去去减少它。\n发言人3: 好的，明白。我这边暂时没有其他问题了。会议助理麻烦最后再播报一下提问方式，谢谢。\n发言人2: 大家好，通过网络端介入的投资者可点击举手连麦等候提问，或在文字交流区提交您的问题。通过电话端进入的投资者，请按新一键等候，先按星号键，再按数字一，谢谢。大家好，通过网络端接入的投资者可点击举手连麦等候提问，或在文字交流区提交您的问题。通过电话端进入的投资者，请按新一键等候，先按星号键在数字一，谢谢。\n发言人2: 大家好，通过网络端介入的投资者可点击举手连麦等候提问，或在文字交流区提交您的问题。通过电话端进入的投资者，请按星一键等候，先按星号键，再按数字一，谢谢。',
 'label': False,
 'fraud_speaker': '',
 'instruction': '\n下面是一段对话文本, 请分析对话内容是否有诈骗风险，以json格式输出你的判断结果(is_fraud: true/false)。\n'}

In [31]:
real_labels, pred_labels = run_test(model, tokenizer, test_data[:10], debug=True)
real_labels, pred_labels


Processing:   0%|          | 0/10 [00:00<?, ?item/s]

受害者王伟: 我需要仔细考虑一下，毕竟这涉及到很大的资金投入。你能给我一些时间吗？
骗子李红: 当然可以，我理解你需要时间来做决定。不过请记住，我们公司的名声是我们最重要的资本，你可以放心地信任我们。


Processing:  10%|█         | 1/10 [00:00<00:04,  2.22item/s]

prediction is {'is_fraud': True}
percent: 0.00%
发言人4: 我们国家在基金在基建上的投资会日益减少。因为我们国家以前在公路城市，地铁的程度是很大的那未来我们在这投资，他解决这个投资，当时更多的是解决什么？那是是一个就业问题是？还有其他拉动其他的问题。


Processing:  20%|██        | 2/10 [00:00<00:02,  2.81item/s]

...

Processing:  90%|█████████ | 9/10 [00:02<00:00,  3.35item/s]

prediction is {'is_fraud': False}
percent: 80.00%
诈骗团伙: 我们手中有大量USDT币出售，支持现场交易和验币。价格相对市场价会更低一些。
李先生: 听起来不错，我有可能会购买一些USDT币。你能给我提供一些详情吗？
诈骗团伙: 当然可以，我们约个时间在大型购物广场见面，到时候可以验币和完成交易。
李先生: 好的，我同意了。我们两天后在大型购物广场见面。
诈骗团伙: 好的，到时候我们会准时赶到。
李先生: 好的，我会准备好的。
诈骗团伙: 交易时我们会用一个数字钱包APP进行“U币”转账，你对此了解吗？
李先生: 没问题，我对这些数字钱包也知道一些。
诈骗团伙: 那太好了，到时候我们会详细教你如何操作。
李先生: 好的，谢谢你们的指导。


Processing: 100%|██████████| 10/10 [00:03<00:00,  3.08item/s]

prediction is {'is_fraud': True}
percent: 90.00%


([True, False, True, False, False, True, False, True, False, True],
 [True, False, True, False, False, False, True, True, False, True])

## 定义计算召回率 和 精确率 的方法
confusion_matrix函数接受一个标签分类值集合和预测分类值集合,返回一个2x2的矩阵包含4个数据,分别是真负,假正,假负,真正的四个统计数字

precision:精确率 表示预测为正的结果中有多少是真的证值,计算公式为: 精确率=真正/(真正+假正)
recall:召回率,表示标签中为正的数据有多少被成功预测召回,计算公式为:召回率 = 真正/(真正+假负)


In [32]:
def precision_recall(true_labels, pred_labels, labels=None, debug=False):
    cm = confusion_matrix(true_labels, pred_labels,labels=labels)
    tn, fp, fn, tp = cm.ravel()
    print(f"tn:{tn}, fp:{fp}, fn:{fn}, tp:{tp}") if debug else None

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    return precision, recall
precision_recall(real_labels, pred_labels, labels=[True, False], debug=False)


(np.float64(0.8), np.float64(0.8))

## 运行评测
将上面的步骤封装到一个evaluate方法中,这样只需要一句代码就能对指定的模型进行评估,并输出结果

In [1]:
def evaluate(model_path, check_point_path, dataset, device='cuda', debug=False):
    model, tokenizer = load_model(model_path, checkpoint_path=check_point_path, device=device)
    true_labels, pred_labels = run_test(model, tokenizer, dataset, device=device, debug=debug)
    precision, recall = precision_recall(true_labels, pred_labels, labels=[True, False], debug=True)
    print(f"precision:{precision}, recall:{recall}")

evaluate(model_path, '', test_data, device='cuda', debug=False)

NameError: name 'model_path' is not defined